# PyTorch: Variable, Gradientes e Grafo Computacional

## Objetivos

Este notebook introduz
- o conceito de `Variables` do PyTorch,
- uma interpretação numérica intuitiva do gradiente, e o
- grafo computacional, utilizado para o cálculo automático do gradiente de uma função.

Um dos principais fundamentos para que o PyTorch seja adequado para deep learning é a sua habilidade de
calcular o gradiente automaticamente a partir da expressões definidas. Essa facilidade é implementada
pelo tipo Variable do PyTorch, que adiciona ao tensor a facilidade de cálculo automático do gradiente pela construção dinâmica do grafo computacional.

## Grafo computacional

```
    y_pred = x * w
    e = y_pred - y
    e2 = e**2
    J = e2.sum()
```

![alt text](https://raw.githubusercontent.com/vcasadei/images/master/GrafoComputacional.png)

Variable possui 3 campos: o dado em si (data), o gradiente (grad) e um apontador (creator) para construir o grafo da backpropagation. Uma expressão utilizada para o cálculo do gradiente exige que todas suas expressões sejam calculadas com Variables, caso contrário não é possível construir o grafo computacional.

![alt text](https://raw.githubusercontent.com/vcasadei/images/master/variables.png)

In [1]:
import torch
from torch.autograd import Variable

## Variable é criada a partir de um tensor e possui as mesmas funcionalidades

In [2]:
y_t = 2 * torch.arange(0.,4.)
y = Variable(y_t); y

tensor([0., 2., 4., 6.])

In [3]:
x = Variable(torch.arange(0.,4.)); x

tensor([0., 1., 2., 3.])

In [4]:
w = Variable(torch.ones(1),requires_grad=True); w

tensor([1.], requires_grad=True)

## Cálculo automático do gradiente da função perda J

Seja a expressão: $$ J = ((x  w) - y)^2 $$

Queremos calcular a derivada de $J$ em relação a $w$.

### Montagem do grafo computacional

In [5]:
# predict (forward)
y_pred = x * w

# cálculo da perda J: loss
e = y_pred - y
e2 = e.pow(2)
J = e2.sum()
J

tensor(14., grad_fn=<SumBackward0>)

## Auto grad - processa o grafo computacional backwards

O `backward()` varre o grafo computacional a partir da variável a ele associada e calcula o gradiente para todas as `Variables` que possuem o atributo `requires_grad` como verdadeiro.
O `backward()` destroi o grafo após sua execução. Isso é intrínsico ao PyTorch pelo fato dele ser uma rede dinâmica.

In [6]:
J.backward()
print(w.grad)

tensor([-28.])


In [7]:
w.grad.data.zero_();

## Interpretação do Gradiente

O gradiente de uma variável final (J) com respeito à outra variável de entrada (w) pode ser interpretado como o quanto a variável final J vai aumentar se houver um pequeno aumento na variável de entrada (w).
Por exemplo suponha que o gradiente seja 28. Isto significa se aumentarmos a variável w de 0.001, então J vai aumentar de 0.028.

In [8]:
eps = 0.001
y_pred = x * (w+eps)
J_new = (y_pred - y).pow(2).sum()
J_new

tensor(13.9720, grad_fn=<SumBackward0>)

In [9]:
print((J_new - J).data.numpy())

-0.027988434


## Back propagation

Uma forma equivalente explícita de calcular o gradiente é fazendo o processamento do backpropagation no grafo computacional, de forma explícita.
Apenas como ilustração.

Função de perda:
$$ J(\hat{y_i},y_i) = \frac{1}{M} \sum_{i=0}^{M-1} (\hat{y_i} - y_i)^2 $$

Gradiente:
$$  \mathbf{\nabla{J_w}} = \frac{2}{M}\mathbf{x^T}(\mathbf{x w^T} - \mathbf{y}) $$

Atualização dos parâmetros pelo gradiente descendente:
$$ \mathbf{w} = \mathbf{w} − \eta (\mathbf{\nabla J_w})^T $$

In [10]:
import numpy as np

dJ = 1.
de2 = dJ * np.ones((4,))
de = de2 * 2 * e.data.numpy()
dy_pred = de
dw = (dy_pred * x.data.numpy()).sum()
print(dJ)
print(de2)
print(de)
print(dw)


1.0
[1. 1. 1. 1.]
[ 0. -2. -4. -6.]
-28.0


## Visualizando o grafo computacional

In [16]:
!pip install graphviz
!pip install git+https://github.com/szagoruyko/pytorchviz

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/szagoruyko/pytorchviz to c:\users\jose.lazaro.n.silva\appdata\local\temp\pip-req-build-c9q7couz
  Resolved https://github.com/szagoruyko/pytorchviz to commit 0adcd83af8aa7ab36d6afd139cabbd9df598edb7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/szagoruyko/pytorchviz 'C:\Users\jose.lazaro.n.silva\AppData\Local\Temp\pip-req-build-c9q7couz'


In [17]:
from torchviz import make_dot, make_dot_from_trace
J = ((w * x) - y).pow(2).sum()
p = {'w':w} # dicionário de parâmetros
out = make_dot(J,params=p)
out

RuntimeError: The size of tensor a (4) must match the size of tensor b (1000) at non-singleton dimension 1

# Exercícios

## Questões

1. Por que numa expressão computacional não é possível misturar `Variable` com tensores?

Não é possível misturar Variable com tensores diretamente porque Variable contém metadados adicionais que permitem a construção do grafo computacional necessário para calcular os gradientes. Misturar Variable com tensores simples pode comprometer a integridade desse grafo, resultando em erros ou cálculos incorretos.

2. O que acontece com o grafo computacional após a execução do `backward()`?

Após a execução do backward(), o PyTorch calcula as derivadas parciais (gradientes) da função de perda em relação a cada um dos parâmetros da rede que possuem requires_grad=True. Nesse caso, para realizar múltiplas iterações de backpropagation, o grafo computacional precisa ser recriado a cada nova execução do forward pass. 

## Atividades

1. Execute um passo de atualização do valor de w, pelo
gradiente descendente. Utilize um fator de aprendizado (*learning rate*) de 0.1
para atualizar o `w`. Após, recalcule a função de perda:

    - w = w - lr * w.grad.data
    - execute a célula 1.3.1 e verifique o quanto que a perda J diminuiu - a célula 1.3.1 é a seguinte print((J_new - J).data.numpy())
    
2. No trecho abaixo, uma rede bastante conhecida, `resnet18` contendo 18 camadas
   é criada, tendo
   como entrada `xin` que é convertida para `Variable`, resultando na saída `y`.
   
   Descomente a linha que cria a vizualização do grafo computacional e execute a
   célula para visualizar o grafo computacional da rede `resnet18`:

In [ ]:
# Dados iniciais
y_t = 2 * torch.arange(0., 4.)
y = Variable(y_t)
x = Variable(torch.arange(0., 4.))
w = Variable(torch.ones(1), requires_grad=True)

# Previsão inicial
y_pred = x * w

# Cálculo da perda inicial J
e = y_pred - y
e2 = e.pow(2)
J = e2.sum()

# Backward para calcular gradientes
J.backward()

# Passo de atualização para w pelo gradiente descendente
lr = 0.1
w.data = w.data - lr * w.grad.data #isso que tu precisa

# Zerando o gradiente acumulado
w.grad.data.zero_()

# Recalculando a previsão e a perda após a atualização
y_pred_new = x * w
J_new = (y_pred_new - y).pow(2).sum()

# Calculando a mudança na perda
print("Mudança na perda J:", (J_new - J).data.numpy())

In [ ]:
%pip install torchvision
from torchvision import models
xin = torch.randn(1,3,224,224)
resnet18 = models.resnet18()
y = resnet18(Variable(xin))
g = make_dot(y, dict(resnet18.named_parameters()))
g

# Aprendizados com este notebook

Com tensores que têm requires_grad=True, o PyTorch consegue identificar as operações feitas nesses tensores e com isso calcular automaticamente os gradientes usando a diferenciação automática(autograd). Isso exclui a necessidade de calcular de forma manuaul os gradientes, simplificando o treinamento de modelos.